In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
from tqdm import tqdm
import sys

In [4]:
def NS_users_crawler(codes, page, page2):
    # User-Agent 설정
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
    result_data = []

    n_ = 0
    for page in range(page, page2+1):
        n_ += 1
        if (n_ % 10 == 0):
            print('================== Page ' + str(page) + ' is done ==================')
        url = "https://finance.naver.com/item/board.naver?code=%s&page=%s" % (codes, str(page))
        # html → parsing
        html = requests.get(url, headers=headers).content
        # 한글 깨짐 방지 decode
        soup = BeautifulSoup(html.decode('euc-kr', 'replace'), 'html.parser')
        table = soup.find('table', {'class': 'type2'})
        tb = table.select('tbody > tr')

        for i in range(2, len(tb)):
            if len(tb[i].select('td > span')) > 0:
                date = tb[i].select('td > span')[0].text
                title = tb[i].select('td.title > a')[0]['title']
                views = tb[i].select('td > span')[1].text
                pos = tb[i].select('td > strong')[0].text
                neg = tb[i].select('td > strong')[1].text
                
                # Extracting the content link URL
                content_url = "https://finance.naver.com" + tb[i].select('td.title > a')[0]['href']
                content_html = requests.get(content_url, headers=headers).content
                content_soup = BeautifulSoup(content_html.decode('euc-kr', 'replace'), 'html.parser')
                content = content_soup.find('div', {'class': 'scr01'}).text.strip()

                result_data.append({'날짜': date, '제목': title, '내용': content, '조회': views, '공감': pos, '비공감': neg})

    # Convert the list of dictionaries to a DataFrame
    result_df = pd.DataFrame(result_data)
    return result_df



In [5]:
data = NS_users_crawler("134790", 1, 100)
data

================== Page 10 is done ==================
================== Page 20 is done ==================
================== Page 30 is done ==================
================== Page 40 is done ==================
================== Page 50 is done ==================
================== Page 60 is done ==================
================== Page 70 is done ==================
================== Page 80 is done ==================
================== Page 90 is done ==================
================== Page 100 is done ==================


,날짜,제목,내용,조회,공감,비공감
0,2023.09.08 14:30,다음주는 더내린다 22000가즈아,ㅋ,6,0,0
1,2023.09.08 08:50,오늘 99.9%,존나게 오른다 밑으로 ㅋ,166,1,1
2,2023.09.08 05:35,앞자리 1자되면,그때 사라,191,1,1
3,2023.09.07 16:08,경고했지요 이거 결국은 2만원이하로 간다고,순진개미들 패가망신주,326,0,1
4,2023.09.07 14:56,ㅋ 살아있니?,ㅋ,331,0,1
...,...,...,...,...,...,...
1995,2022.08.22 15:19,ㅋㅋㅋㅋㅋㅋㅋ,개굿 잘 팔았네,87,1,0
1996,2022.08.22 15:19,아휴 미련 곰탱이들아,다나갔냐,138,1,1
1997,2022.08.22 15:19,이게 주식이다,고속운지,129,3,1
1998,2022.08.22 15:19,신풍제약임?,무섭다,107,1,1


In [6]:
data.to_csv('시디즈_토론.csv',encoding='utf-8-sig',index=False)